In [3]:
# !pip install gdown
# !gdown --folder https://drive.google.com/drive/folders/18fbeQOzN4BMn09LPnFgWflhTP-r9JJrc?usp=sharing
# !pip install -r requirements.txt

In [4]:
import pytorch_lightning as pl
import torch

from torch import Tensor, nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import EarlyStopping

from engine.data import get_data_loader
from engine.metrics import accuracy
from engine.model_base import LightningBaseModule

torch.set_float32_matmul_precision("medium")

In [5]:
train_loader = get_data_loader("train")
val_loader = get_data_loader("val")
test_loader = get_data_loader("test")

In [6]:
EMBEDDING_SIZE = 80


class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention = nn.MultiheadAttention(EMBEDDING_SIZE, 10)
        self.activation = nn.ReLU()

    def forward(self, X):
        output, _ = self.attention(X, X, X)
        return self.activation(output)


class AttentionBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.attentions = nn.Sequential(*[Attention() for _ in range(3)])
        self.layer_norm = nn.LayerNorm(EMBEDDING_SIZE)

    def forward(self, X):
        output = self.attentions(X)
        output = self.layer_norm(output)
        return output + X


class AttentionModel(LightningBaseModule):
    def __init__(self):
        super().__init__()
        self.attention_1 = nn.Sequential(*[AttentionBlock() for _ in range(5)])
        self.attention_2 = nn.Sequential(*[AttentionBlock() for _ in range(5)])
        self.flatten = nn.Flatten()
        self.dense = nn.Sequential(
            nn.Linear(8000, 1000),
            nn.ReLU(),
            nn.Linear(1000, 22),
            nn.Softmax(dim=1),
        )
        self.loss = nn.CrossEntropyLoss()

    def forward(self, X):
        output = self.attention_1(X)
        output = self.attention_2(output + X)
        output = self.flatten(output)
        output = self.dense(output)
        return output

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = ReduceLROnPlateau(optimizer, patience=5)
        return [optimizer], [
            {
                "scheduler": scheduler,
                "interval": "epoch",
                "monitor": "val_accuracy",
                "frequency": 1,
            }
        ]

In [7]:
callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        mode="max",
        patience=10,
        min_delta=1e-4,
    )
]

model = AttentionModel().cuda()
trainer = pl.Trainer(max_epochs=5, callbacks=callbacks)
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | attention_1 | Sequential       | 389 K 
1 | attention_2 | Sequential       | 389 K 
2 | flatten     | Flatten          | 0     
3 | dense       | Sequential       | 8.0 M 
4 | loss        | CrossEntropyLoss | 0     
-------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.209    Total estimated model params size (MB)


/home/dawid/miniconda3/envs/dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/dawid/miniconda3/envs/dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 1:  19%|█▉        | 157/837 [00:07<00:33, 20.37it/s, v_num=25, train_step_loss=2.620, val_step_loss=2.570]

/home/dawid/miniconda3/envs/dl/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
